In [1]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from lohrasb.best_estimator import BaseModel
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from sklearn.metrics import f1_score, accuracy_score, make_scorer
from lightgbm import *


2023-07-01 20:11:13,790 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-07-01 20:11:13,797 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-07-01 20:11:13,799 :: matplotlib :: interactive is False
2023-07-01 20:11:13,800 :: matplotlib :: platform is darwin


# Example: Use Adult Data Set (a classification problem)
  
https://archive.ics.uci.edu/ml/datasets/Adult

# Part 1: Use BestModel in sklearn pipeline


In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# column names
col_names=["age", "workclass", "fnlwgt" , "education" ,"education-num",
"marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
"native-country","label"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Define labels


In [3]:
data.loc[data['label']=='<=50K','label']=0
data.loc[data['label']==' <=50K','label']=0

data.loc[data['label']=='>50K','label']=1
data.loc[data['label']==' >50K','label']=1

data['label']=data['label'].astype(int)

#### Train test split

In [4]:
X = data.loc[:, data.columns != "label"]
y = data.loc[:, data.columns == "label"]


X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)


#### Find feature types for later use

In [5]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#### Define estimator and set its arguments 


In [6]:
estimator = LGBMClassifier()
estimator_params = {
        "max_depth": [6,30],
        "learning_rate": [0.001,0.01, 0.1],
        "n_estimators": [100,200,1000],
        "boosting_type": ["gbdt"],

    }
    

In [7]:

kwargs = {  # params for fit method or fit_params 
            'fit_random_kwargs' :{
            'sample_weight':None,
            },
            # params for RandomSearchCV 
            'random_search_kwargs' : {
            'estimator':estimator,
            'param_distributions':estimator_params,
            'scoring' :'f1',
            'verbose':3,
            'n_jobs':-1,
            'cv':KFold(2),
            'n_iter':10,
            'refit':True,
            }
            }



In [8]:

obj = BaseModel().optimize_by_randomsearchcv(
    kwargs=kwargs
 )

#### Build sklearn pipeline

In [9]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model
            ('obj', obj)

 ])


#### Run Pipeline

In [10]:
pipeline.fit(X_train,y_train.values.ravel())
y_pred = pipeline.predict(X_test)


Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV 2/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=30, n_estimators=100;, score=0.000 total time=   0.5s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=30, n_estimators=100;, score=0.000 total time=   0.5s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=30, n_estimators=100;, score=0.696 total time=   0.5s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=30, n_estimators=100;, score=0.705 total time=   0.5s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=30, n_estimators=200;, score=0.660 total time=   0.8s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=30, n_estimators=200;, score=0.652 total time=   0.8s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=30, n_estimators=200;, score=0.000 total time=   0.7s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=30, n_estimators=200;, score=0.000 total time=   0.7s
[

#### Check performance of the pipeline

In [11]:
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))


F1 score : 
0.7190357439733999
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8158
           1       0.78      0.67      0.72      2588

    accuracy                           0.87     10746
   macro avg       0.84      0.80      0.82     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7664  494]
 [ 858 1730]]


# Part 2:  Use BestModel as a standalone estimator 

In [12]:
X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)

#### Transform features to make them ready for model input

In [13]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [14]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [15]:
obj.fit(X_train,y_train.values.ravel())
y_pred = obj.predict(X_test)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV 1/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=30, n_estimators=100;, score=0.000 total time=   0.3s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=30, n_estimators=100;, score=0.696 total time=   0.3s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=30, n_estimators=100;, score=0.705 total time=   0.3s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=30, n_estimators=100;, score=0.000 total time=   0.3s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=6, n_estimators=200;, score=0.657 total time=   0.6s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=6, n_estimators=200;, score=0.648 total time=   0.6s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=30, n_estimators=200;, score=0.660 total time=   0.6s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=30, n_estimators=200;, score=0.652 total time=   0.6s
[CV 1

#### Check performance of the model

In [16]:
print('accuracy score score : ')
print(accuracy_score(y_test,y_pred))
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))

accuracy score score : 
0.8741857435324772
F1 score : 
0.7190357439733999
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8158
           1       0.78      0.67      0.72      2588

    accuracy                           0.87     10746
   macro avg       0.84      0.80      0.82     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7664  494]
 [ 858 1730]]


In [17]:
obj.get_best_estimator()

LGBMClassifier(max_depth=30)

In [18]:
obj.best_estimator

LGBMClassifier(max_depth=30)

#### Get fitted randomized search object and its attributes

In [19]:
RandomSearchObj = obj.get_optimized_object()
RandomSearchObj.cv_results_

{'mean_fit_time': array([2.11612499, 0.18156648, 1.59549356, 1.92535949, 0.22555506,
        0.37494814, 1.73480213, 0.23203754, 0.42063749, 0.20732236]),
 'std_fit_time': array([0.00757682, 0.01568961, 0.00621438, 0.01926565, 0.00424111,
        0.0124768 , 0.0201093 , 0.00796533, 0.00585258, 0.00321746]),
 'mean_score_time': array([3.40414405, 0.09584236, 3.45511556, 3.65223122, 0.10418904,
        0.19641626, 3.0220654 , 0.17844296, 0.17994237, 0.08740711]),
 'std_score_time': array([0.08468509, 0.00179648, 0.23433661, 0.09037399, 0.00112712,
        0.00260746, 0.0142715 , 0.02060413, 0.00128841, 0.00283599]),
 'param_n_estimators': masked_array(data=[1000, 100, 1000, 1000, 100, 200, 1000, 100, 200, 100],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[30, 6, 30, 6, 30, 6, 6, 30, 30, 30],
              mask=[False, False, False, F